# EDE - Interplanetary Probes - Radar Calculation

First, import useful standard packages

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy.units as u

Next, rearrange the radar SNR equation, as it was given to us by Michael Busch, to solve for distance. This is also the form in which it is presented in the Ostro paper.

In [10]:
def squiggle_interplanetary_probe_radar_ede_distance(target_diameter,
                                            p_rot,
                                            t_integration,
                                            SNR,
                                            telescope_effective_area,
                                            transmitter_power):
    distance = (target_diameter**(3/2) * p_rot**(1/2) * t_integration**(1/2) * SNR**(-1) * transmitter_power * telescope_effective_area**(2))**(1/4)
    
    return distance.si

Try putting in some random numbers to test that the units work out and we get a distance

In [11]:
interplanetary_probe_radar_ede_distance(1 * u.m,
                                        1 * u.day,
                                        1 * u.second,
                                        10,
                                        100 * u.m**2,
                                        10 * u.W)

<Quantity 41.40608962 kg(1/4) m(15/8) / s(1/2)>

Ahaha, so, we don't get a distance... that means that the SNR estimate equation is missing some constants or something. I looked up some lecture notes to find the full version. The PDF I dug up is in this directory, and contains the following equation:

Note that I take kT_0 to be 4 * 10^-21 W/Hz, just like in the lecture notes, corresponding to a reference temperature of 290 K.

In [17]:
def interplanetary_probe_radar_ede_distance(peak_transmit_power,
                                            gain_transmit,
                                            gain_receive,
                                            radar_wavelength,
                                            radar_cross_section,
                                            effective_noise_bandwidth,
                                            radar_noise_figure,
                                            other_losses,
                                            snr):
    
    kT_0 = 4 * 10**-21 * u.W / u.Hz
    distance = (peak_transmit_power * gain_transmit * gain_receive * radar_wavelength**2 * radar_cross_section / ((4 * np.pi)**3 * snr * kT_0 * effective_noise_bandwidth * radar_noise_figure * other_losses))**(1/4)
    
    return distance.si

Trying this with some random numbers to check units.

In [18]:
interplanetary_probe_radar_ede_distance(1 * u.W,
                                        2,
                                        2,
                                        1 * u.m,
                                        4 * u.m**2,
                                        1 * u.Hz,
                                        5,
                                        1,
                                        10)

<Quantity 14169.84497014 m>

Okay, it's a distance this time!!! Next step, I think, is to read through the rest of the radar PDF and take notes about each term. Once we understand it, we can plug in the real numbers for a few case studies (Arecibo-to-mylar balloon, Arecibo-to-well-aligned-ISS) and call it a day!